In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras 
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input

from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.models import load_model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from keras.applications.xception import Xception
from tensorflow.keras.applications.densenet import DenseNet169, DenseNet201, DenseNet121
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.inception_v3 import InceptionV3


tf.__version__

In [3]:
# Part 1 - Data Preprocessing

# Preprocessing the Training set

# train_datagen = ImageDataGenerator(rescale = 1./255,
#                                    shear_range = 0.2,
#                                    zoom_range = 0.2,
#                                    horizontal_flip = True,
#                          vertical_flip=True,
#                          width_shift_range = 0.1,
#                          height_shift_range = 0.1,)
# training_set = train_datagen.flow_from_directory('../input/wastesegg/waste_seg/DATASET/TRAIN',
#                                                  target_size = (224,224),
#                                                  batch_size = 20,
#                                                  shuffle = True,
#                                                  class_mode = 'categorical')

# valid_datagen = ImageDataGenerator(rescale = 1./255)
# valid_set = valid_datagen.flow_from_directory('../input/wastesegg/waste_seg/DATASET/VAL',
#                                             target_size = (224, 224)
#                                             )



# # Preprocessing the Test set

# test_datagen = ImageDataGenerator(rescale = 1./255)
# test_set = test_datagen.flow_from_directory('../input/wastesegg/waste_seg/DATASET/TEST',
#                                             target_size = (224, 224)
#                                             )



# ntest_datagen = ImageDataGenerator(rescale = 1./255)


# ntest_set = ntest_datagen.flow_from_directory('../input/wastesegg/waste_seg/DATASET/TEST/N',
#                                             target_size = (224, 224)
#                                            )

# otest_datagen = ImageDataGenerator(rescale = 1./255)
# otest_set = otest_datagen.flow_from_directory('../input/wastesegg/waste_seg/DATASET/TEST/O',
#                                             target_size = (224, 224)
#                                            )

# rtest_datagen = ImageDataGenerator(rescale = 1./255)
# rtest_set = rtest_datagen.flow_from_directory('../input/wastesegg/waste_seg/DATASET/TEST/R',
#                                             target_size = (224, 224)
#                                            )


nDir = '../input/segwaste/seg_data/DATASET/'
oDir = '../input/segwaste/seg_data/DATASET/'
rDir = '../input/segwaste/seg_data/DATASET/'

nImagePaths = os.listdir(nDir+'N/')
oImagePaths = os.listdir(oDir+'O/')
rImagePaths = os.listdir(rDir+'R/')

# print(nImagePaths)
# print(oImagePaths)
# print(rImagePaths)

print(len(nImagePaths))
print(len(oImagePaths))
print(len(rImagePaths))


In [ ]:
# type(train_datagen), type(training_set), type(training_set), type(test_set)
#for _ in range(10):
 #   i = test_set.next()
  #  print(len(i), i)

nImagePaths.sort()
oImagePaths.sort()
rImagePaths.sort()


# print(nImagePaths)
# print(oImagePaths)
# print(rImagePaths)

In [ ]:
import cv2
import matplotlib.pyplot as plt

nImgs = list()
oImgs = list()
rImgs = list()
nLabels = list()
oLabels = list()
rLabels = list()

import gc
gc.collect()

for i in range(len(nImagePaths)):
    nImgs.append(cv2.resize(cv2.imread(nDir+'N/'+nImagePaths[i]), (128,128)))
    #nImgs.append(cv2.rescale(cv2.imread(nDir+'N/'+nImagePaths[i]), (1./255)))
    nLabels.append(0)
gc.collect()
for i in range(len(oImagePaths)):
    oImgs.append(cv2.resize(cv2.imread(oDir+'O/'+oImagePaths[i]), (128,128)))
    oLabels.append(1)
gc.collect()
for i in range(len(rImagePaths)):
    rImgs.append(cv2.resize(cv2.imread(rDir+'R/'+rImagePaths[i]), (128,128)))
    rLabels.append(2)
gc.collect()
#print(nImgs)
# print(nLabels)
# print(oLabels)
# print(rLabels)

In [ ]:
gc.collect()
allImages = nImgs + oImgs + rImgs
allLabels = nLabels + oLabels + rLabels

print(len(allImages))
print(len(allLabels))
# print(allLabels)
# print(allImages)

In [ ]:
# for i in range(len(allImages)):
#     print(allImages[i].shape)
    
import gc
gc.collect()    
allImages = np.stack(allImages, axis = 0)
allLabels = np.array(allLabels)
print(allLabels)
#print(allImages)

print(allImages.shape)
print(allLabels.shape)

In [ ]:
from sklearn.model_selection import train_test_split
from PIL import Image
from tensorflow.keras.layers import BatchNormalization
from sklearn.metrics import confusion_matrix
from tensorflow.keras import utils as np_utils
#from keras.utils import to_categorical

gc.collect()

xTrain,xTest,yTrain,yTest = train_test_split(allImages,np.array(allLabels), test_size=0.20, random_state = 30, stratify = allLabels)
gc.collect()
xPTrain,xVal,yPTrain,yVal = train_test_split(xTrain,yTrain, test_size=0.20, random_state = 30)

#yPTrain = to_categorical(yPTrain, 3)
# yVal = keras.utils.to_categorical(yVal, 3)
# yTest = keras.utils.to_categorical(yTest, 3)
import gc
gc.collect()

print(xPTrain.shape)
print(yPTrain.shape)
print(xVal.shape)
print(yVal.shape)
print(xTest.shape)
print(yTest.shape)
gc.collect()

In [ ]:
gc.collect()
gc.collect()

In [ ]:
# image Augmentation

import random

newXPTrain = list()
newYPTrain =list()

gc.collect()

for i in range(len(xPTrain)):
    # Add the original image
    newXPTrain.append(xPTrain[i].reshape(128,128,3))
    newYPTrain.append(yPTrain[i])
    # Add the horizontally flipped image
# gc.collect()   
# # for i in range(len(xPTrain)):
# #     newXPTrain.append(cv2.flip(xPTrain[i].reshape(128,128,3),1))
# #     newYPTrain.append(yPTrain[i])
# # gc.collect()    
# for i in range(len(xPTrain)):
#     newXPTrain.append(cv2.flip(xPTrain[i].reshape(128,128,3),0))
#     newYPTrain.append(yPTrain[i])
    
gc.collect()
    
newXPTrain = np.stack(newXPTrain, axis=0)

gc.collect()
newXPTrain = newXPTrain.reshape(len(newXPTrain), 128,128,3)
newYPTrain = np.array(newYPTrain)
gc.collect()
xVal=xVal.reshape(len(xVal), 128,128,3)
yVal= np.array(yVal)
#newYPTrain = to_categorical(yPTrain, 3)
gc.collect()
print(newXPTrain.shape)
print(newYPTrain.shape)
print(xVal.shape)
print(yVal.shape)
gc.collect()

In [ ]:
gc.collect()
gc.collect()

In [ ]:
# Part 2 - Building the CNN
input_tensor = Input(shape=(128, 128, 3))
incept_weights = "../input/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
# base_model = tf.keras.applications.Xception
base_model1 = tf.keras.applications.InceptionV3(
    input_tensor=input_tensor,
    include_top  = False,
    weights      = 'imagenet',
)
base_model1.load_weights(incept_weights)
x1 = base_model1.output
print(base_model1.input)
print(base_model1.output)

gc.collect()
input_tensor = Input(shape=(128, 128, 3))
vgg_weights = "../input/vgg16model/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
base_model2 = tf.keras.applications.vgg16.VGG16(
    input_tensor=input_tensor,
    include_top  = False,
    weights      = 'imagenet',
)
base_model2.load_weights(vgg_weights)
x2 = base_model2.output
print(base_model2.input)
print(base_model2.output)

gc.collect()
input_tensor = Input(shape=(128, 128, 3))
res_weights = "../input/resnet/resnet152v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
base_model3 = tf.keras.applications.resnet_v2.ResNet152V2(
    input_tensor=input_tensor,
    include_top  = False,
    weights      = 'imagenet',
)
base_model3.load_weights(res_weights)
x3 = base_model3.output
print(base_model3.input)
print(base_model3.output)

gc.collect()
input_tensor = Input(shape=(128, 128, 3))
mob_weights = "../input/mobilenet/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_192_no_top.h5"
base_model4 = tf.keras.applications.MobileNetV2(
    input_tensor=input_tensor,
    include_top  = False,
    weights      = 'imagenet',
)
base_model4.load_weights(mob_weights)
x4 = base_model4.output
print(base_model4.input)
print(base_model4.output)

gc.collect()
input_tensor = Input(shape=(128, 128, 3))
dense_weights = "../input/densenet201/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5"
base_model5 = tf.keras.applications.densenet.DenseNet201(
    input_tensor=input_tensor,
    include_top  = False,
    weights      = 'imagenet',
)
base_model5.load_weights(dense_weights)
x5 = base_model5.output
print(base_model5.input)
print(base_model5.output)
gc.collect()


#tf.keras.applications.InceptionResNetV2
#tf.keras.applications.MobileNetV2

In [ ]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
#base_model.trainable = False
x1 = tf.keras.layers.GlobalAveragePooling2D()
x2 = tf.keras.layers.GlobalAveragePooling2D()
x3 = tf.keras.layers.GlobalAveragePooling2D()
x4 = tf.keras.layers.GlobalAveragePooling2D()
x5 = tf.keras.layers.GlobalAveragePooling2D()
#x = Dense(1024, activation='relu')(x)
predictions1 = tf.keras.layers.Dense(3,activation="softmax")
predictions2 = tf.keras.layers.Dense(3,activation="softmax")
predictions3 = tf.keras.layers.Dense(3,activation="softmax")
predictions4 = tf.keras.layers.Dense(3,activation="softmax")
predictions5 = tf.keras.layers.Dense(3,activation="softmax")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense
model1 = Sequential([base_model1,
                   x1,
                    predictions1
                ])
print(model1)
model1.summary()

model2 = Sequential([base_model2,
                   x2,
                    predictions2
                ])
print(model2)
model2.summary()

model3 = Sequential([base_model3,
                   x3,
                    predictions3
                ])
print(model3)
model3.summary()

model4 = Sequential([base_model4,
                   x4,
                    predictions4
                ])
print(model4)
model4.summary()

model5 = Sequential([base_model5,
                   x5,
                    predictions5
                ])
print(model5)
model5.summary()

In [ ]:
# base_model1.summary()
# base_model2.summary()

In [ ]:
#base_model.trainable = False
for layer in base_model1.layers:
    layer.trainable = False
    
for layer in base_model2.layers:
    layer.trainable = False
    
for layer in base_model3.layers:
    layer.trainable = False
    
for layer in base_model4.layers:
    layer.trainable = False
    
for layer in base_model5.layers:
    layer.trainable = False

In [ ]:
base_model1.summary()
base_model2.summary()
base_model3.summary()
base_model4.summary()
base_model5.summary()

In [ ]:
gc.collect()
def get_callbacks(name):
    return[
        tfdocs.modeling.EpochDots(),
        tf.keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_crossentropy',patience=10),
    ]

In [ ]:
gc.collect()
base_learning_rate  = 0
model1.compile(optimizer=tf.keras.optimizers.Adam(), loss="sparse_categorical_crossentropy",metrics=[tf.keras.losses.SparseCategoricalCrossentropy(name='categorical_crossentropy'),'accuracy'])
model2.compile(optimizer=tf.keras.optimizers.Adam(), loss="sparse_categorical_crossentropy",metrics=[tf.keras.losses.SparseCategoricalCrossentropy(name='categorical_crossentropy'),'accuracy'])
model3.compile(optimizer=tf.keras.optimizers.Adam(), loss="sparse_categorical_crossentropy",metrics=[tf.keras.losses.SparseCategoricalCrossentropy(name='categorical_crossentropy'),'accuracy'])
model4.compile(optimizer=tf.keras.optimizers.Adam(), loss="sparse_categorical_crossentropy",metrics=[tf.keras.losses.SparseCategoricalCrossentropy(name='categorical_crossentropy'),'accuracy'])
model5.compile(optimizer=tf.keras.optimizers.Adam(), loss="sparse_categorical_crossentropy",metrics=[tf.keras.losses.SparseCategoricalCrossentropy(name='categorical_crossentropy'),'accuracy'])
gc.collect()

In [ ]:
gc.collect()
gc.collect()

In [ ]:
# history = model1.fit(x = training_set, epochs= 1, validation_data = valid_set)
# history = model2.fit(x = training_set, epochs= 1, validation_data = valid_set)
# from tensorflow.keras.callbacks import EarlyStopping
# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50, restore_best_weights=True)

# chkpt_cb = tf.keras.callbacks.ModelCheckpoint(filepath='model_lr01_amsgrad_s1_b2.h5',
#                                               monitor='val_loss',
#                                               verbose=1,
#                                               save_best_only=True)

# from keras.utils import to_categorical

# # make the model and load the training dataset.

# newYPTrain = to_categorical(newYPTrain,3)
# yVal = to_categorical(yVal,3)

# import gc
gc.collect()
history1 = model1.fit(newXPTrain, newYPTrain, batch_size= 10, epochs= 40, verbose=1, 
                     validation_data = (xVal, yVal))
# , class_weight = class_weight
             
gc.collect()
history2 = model2.fit(newXPTrain, newYPTrain, batch_size= 10, epochs= 40, verbose=1, 
                     validation_data = (xVal, yVal))
gc.collect()

history3 = model3.fit(newXPTrain, newYPTrain, batch_size= 10, epochs= 40, verbose=1, 
                     validation_data = (xVal, yVal))
gc.collect()

history4 = model4.fit(newXPTrain, newYPTrain, batch_size= 10, epochs= 40, verbose=1, 
                     validation_data = (xVal, yVal))
gc.collect()

history5 = model5.fit(newXPTrain, newYPTrain, batch_size= 10, epochs= 40, verbose=1, 
                     validation_data = (xVal, yVal))
gc.collect()
              
             # , class_weight = class_weight
        

In [ ]:
model1.save('model1', save_format='h5')
model2.save('model2', save_format='h5')
model3.save('model3', save_format='h5')
model4.save('model4', save_format='h5')
model5.save('model5', save_format='h5')

In [ ]:
# classes =["N","O","R"]

In [ ]:
model1.load_weights('../input/trained-models/model1')
model2.load_weights('../input/trained-models/model2')
model3.load_weights('../input/trained-models/model3')
model4.load_weights('../input/trained-models/model4')
model5.load_weights('../input/trained-models/model5')

In [ ]:


#model.evaluate(test_set)
# print(allLabels)
print(len(allLabels))
# pred1=model1.predict_generator(test_set)
# print(pred1)
# print(len(pred1))
# p1=np.argmax(pred1, axis=1)
# print(p1)
# pred2=model2.predict_generator(test_set)
# print(pred2)
# p2=np.argmax(pred2, axis=1)
# print(p2)
#pred3=model3.predict_proba(x_test)
# pred1= model1.predict(xTest)
# print(pred1)
# pred2= model2.predict(xTest)
# print(pred2)


# finalpred1=(pred1+pred2)/2
# print(finalpred1)
# predict1 = np.argmax(finalpred1, axis=1)
# print(predict1)
gc.collect()
pred1= model1.predict(xTest)
print(pred1)
p1=np.argmax(pred1,axis=1)
print(p1)
gc.collect()
pred2= model2.predict(xTest)
print(pred2)
p2=np.argmax(pred2,axis=1)
print(p2)
gc.collect()
pred3= model3.predict(xTest)
print(pred3)
p3=np.argmax(pred3,axis=1)
print(p3)
gc.collect()
pred4= model4.predict(xTest)
print(pred4)
p4=np.argmax(pred4,axis=1)
print(p4)
gc.collect()
pred5= model5.predict(xTest)
print(pred5)
p5=np.argmax(pred5,axis=1)
print(p5)
gc.collect()

# predict1 = np.argmax(pred2, axis=1)


# finalpred1=(pred2+pred4+pred5)/3
finalpred1=(pred1+pred2+pred3+pred4+pred5)/5
print(finalpred1)
predict1 = np.argmax(finalpred1, axis=1)
print(predict1)

import numpy as np
from scipy import stats

# final_pred2 = np.array([p2, p4, p5])
final_pred2 = np.array([p1, p2, p3, p4, p5])
pr2 = stats.mode(final_pred2)
print(pr2)
p = np.array([pr2[0]])
print(p)
print(p.shape)

# final_pred3 = (pred3*0.306)+(pred4*0.345)+(pred5*0.347)

final_pred3 = (pred1*0.1651)+(pred2*0.2256)+(pred3*0.1865)+(pred4*0.2089)+(pred5*0.2138)
# final_pred3 = (pred2*0.344)+(pred4*0.326)+(pred5*0.328)
print(final_pred3)
predict3 = np.argmax(final_pred3, axis=1)
print(predict3)
print(predict3.shape)


# import statistics
# #print(predict1)
# final_pred2 = np.array([])
# # final_pred2 = np.array().reshape(xTest, 5)
# for i in range(0,len(xTest)):
#     final_pred2 = np.append(final_pred2, statistics.mode([p4[i], p5[i]]))
# print(final_pred2)
# # predict2 = np.argmax(final_pred2, axis=1)
# predict2=final_pred2
# print(predict2)



print(len(predict1))

# binProb = pred2.ravel()
# predict1 = list()
# for i in range(len(binProb)):
#   if binProb[i] > 0.66:
#     predict1.append(1)
#   else:
#     predict1.append(0)





In [ ]:
print(predict1.shape)
print(p.shape)
print(pr2[0].shape)

predict2=np.reshape(p,7866)
print(predict2.shape)

In [ ]:
with open("model_Pred1.txt", "w") as output:
    output.write(str(predict1))
    
with open("model_Pred2.txt", "w") as output:
    output.write(str(predict2))
    
with open("model_Pred3.txt", "w") as output:
    output.write(str(predict3))

#print(training_set.class_indices)
#cnn.save("mobilenetmodel1.h5")

from sklearn.metrics import classification_report

target_names = ['N', 'O', 'R']
print(classification_report(list(yTest), predict1, target_names=target_names))
target_names = ['N', 'O', 'R']
print(classification_report(list(yTest), predict2, target_names=target_names))
target_names = ['N', 'O', 'R']
print(classification_report(list(yTest), predict3, target_names=target_names))

In [ ]:
from sklearn import metrics

cm1 = confusion_matrix(yTest, predict1)
print(cm1)

TP1 = cm1[0][0]
FP1 = cm1[0][1]
FN1 = cm1[1][0]
TN1 = cm1[1][1]

print(TP1)
print(FP1)
print(FN1)
print(TN1)

sens1 = TP1 / (TP1 + FN1)
spec1 = TN1 / (TN1 + FP1)
prec1 = TP1 / (TP1 + FP1)
rec1 = TP1 / (TP1 + FN1)
f11 = (2*prec1*rec1)/(prec1+rec1)
acc1 = (TP1 + TN1) / (TP1 + FP1 + TN1 + FN1)
er1 = 1-acc1
print('Accuracy: ', acc1)
print('Sensitivity: ', sens1)
print('Specificity: ', spec1)
print('Precision: ', prec1)
print('Recall: ', rec1)
print('F1 Score: ', f11)
print('Error rate:',er1)
binProb1 = predict1.ravel()
fpr1, tpr1, thresholds1 = metrics.roc_curve(yTest, binProb1, pos_label=1)
# print(fpr1)
# print(tpr1)
# print(thresholds1)
print('AUC: ',metrics.auc(fpr1, tpr1))

cm2 = confusion_matrix(yTest, predict2)
print(cm2)

TP2 = cm2[0][0]
FP2 = cm2[0][1]
FN2 = cm2[1][0]
TN2 = cm2[1][1]

print(TP2)
print(FP2)
print(FN2)
print(TN2)

sens2 = TP2 / (TP2 + FN2)
spec2 = TN2 / (TN2 + FP2)
prec2 = TP2 / (TP2 + FP2)
rec2 = TP2 / (TP2 + FN2)
f12 = (2*prec2*rec2)/(prec2+rec2)
acc2 = (TP2 + TN2) / (TP2 + FP2 + TN2 + FN2)
er2 = 1-acc2
print('Accuracy: ', acc2)
print('Sensitivity: ', sens2)
print('Specificity: ', spec2)
print('Precision: ', prec2)
print('Recall: ', rec2)
print('F1 Score: ', f12)
print('Error rate:',er2)
binProb2 = predict2.ravel()
fpr2, tpr2, thresholds2 = metrics.roc_curve(yTest, binProb2, pos_label=1)
print('AUC: ',metrics.auc(fpr2, tpr2))

cm3 = confusion_matrix(yTest, predict3)
print(cm3)

TP3 = cm3[0][0]
FP3 = cm3[0][1]
FN3 = cm3[1][0]
TN3 = cm3[1][1]

print(TP3)
print(FP3)
print(FN3)
print(TN3)

sens3 = TP3 / (TP3 + FN3)
spec3 = TN3 / (TN3 + FP1)
prec3 = TP3 / (TP1 + FP1)
rec3 = TP3 / (TP3 + FN3)
f13 = (2*prec3*rec3)/(prec3+rec3)
acc3 = (TP3 + TN3) / (TP3 + FP3 + TN3 + FN3)
er3 = 1-acc3
print('Accuracy: ', acc3)
print('Sensitivity: ', sens3)
print('Specificity: ', spec3)
print('Precision: ', prec3)
print('Recall: ', rec3)
print('F1 Score: ', f13)
print('Error rate:',er3)
binProb3 = predict3.ravel()
fpr3, tpr3, thresholds3 = metrics.roc_curve(yTest, binProb3, pos_label=1)
print('AUC: ',metrics.auc(fpr3, tpr3))

In [ ]:
import matplotlib.pyplot as plt
# Accuracy/validation plots
h1 = history1.history
fig = plt.figure(figsize = (13, 4))

plt.subplot(121)
plt.plot(h1['accuracy'], label = 'train_acc')
plt.plot(h1['val_accuracy'], label = 'val_acc')
plt.legend()
plt.grid(False)
plt.title(f'Training and Validation accuracy')

plt.subplot(122)
plt.plot(h1['loss'], label = 'train_loss')
plt.plot(h1['val_loss'], label = 'val_loss')
plt.legend()
plt.grid(False)
plt.title(f'Training and Validation loss')


h2 = history2.history
fig = plt.figure(figsize = (13, 4))

plt.subplot(121)
plt.plot(h2['accuracy'], label = 'train_acc')
plt.plot(h2['val_accuracy'], label = 'val_acc')
plt.legend()
plt.grid(False)
plt.title(f'Training and Validation accuracy')

plt.subplot(122)
plt.plot(h2['loss'], label = 'train_loss')
plt.plot(h2['val_loss'], label = 'val_loss')
plt.legend()
plt.grid(False)
plt.title(f'Training and Validation loss')

h3 = history3.history
fig = plt.figure(figsize = (13, 4))

plt.subplot(121)
plt.plot(h3['accuracy'], label = 'train_acc')
plt.plot(h3['val_accuracy'], label = 'val_acc')
plt.legend()
plt.grid(False)
plt.title(f'Training and Validation accuracy')

plt.subplot(122)
plt.plot(h3['loss'], label = 'train_loss')
plt.plot(h3['val_loss'], label = 'val_loss')
plt.legend()
plt.grid(False)
plt.title(f'Training and Validation loss')

h4 = history4.history
fig = plt.figure(figsize = (13, 4))

plt.subplot(121)
plt.plot(h4['accuracy'], label = 'train_acc')
plt.plot(h4['val_accuracy'], label = 'val_acc')
plt.legend()
plt.grid(False)
plt.title(f'Training and Validation accuracy')

plt.subplot(122)
plt.plot(h4['loss'], label = 'train_loss')
plt.plot(h4['val_loss'], label = 'val_loss')
plt.legend()
plt.grid(False)
plt.title(f'Training and Validation loss')

h5 = history5.history
fig = plt.figure(figsize = (13, 4))

plt.subplot(121)
plt.plot(h5['accuracy'], label = 'train_acc')
plt.plot(h5['val_accuracy'], label = 'val_acc')
plt.legend()
plt.grid(False)
plt.title(f'Training and Validation accuracy')

plt.subplot(122)
plt.plot(h5['loss'], label = 'train_loss')
plt.plot(h5['val_loss'], label = 'val_loss')
plt.legend()
plt.grid(False)
plt.title(f'Training and Validation loss')

In [4]:
import matplotlib.pyplot as plt
Accuracy = [93.099, 93.664, 93.625, 92.92, 93.209, 93.324]
# fig = plt.figure(figsize = (, 5))
plt.plot(Accuracy, label = "Accuracy")
plt.xlabel('Proposed Approaches')
plt.ylabel('Accuracy')
# # plt.subplot(121)
# plt.plot(h1['accuracy'], label = 'train_acc')
# plt.plot(h1['val_accuracy'], label = 'val_acc')
# plt.legend()
# plt.grid(False)
# plt.title(f'Training and Validation accuracy')

In [3]:
import matplotlib.pyplot as plt
# fig = plt.figure(figsize = (8, 20))
fig, ax1 = plt.subplots(1,1, figsize=(6,5))

x1 = [0,1,2,3,4,5]
squad = ['Accuracy-based','Precision-based','F1-based','Recall-based', 'Specificity-based', 'Performance-based']

ax1.set_xticks(x1)
ax1.set_xticklabels(squad, minor=False, rotation=45)
# fig = plt.figure(figsize = (5, 5))
Accuracy = [93.099, 93.664, 93.625, 92.92, 93.209, 93.324]

# plt.plot(Accuracy, label = "Accuracy")
plt.bar(squad, Accuracy)
plt.ylim(92,94)
plt.xlabel('Proposed Approaches')
plt.ylabel('Accuracy')



fig, ax1 = plt.subplots(1,1, figsize=(6,5))

x1 = [0,1,2,3,4,5]
squad = ['Accuracy-based','Precision-based','F1-based','Recall-based', 'Specificity-based', 'Performance-based']

ax1.set_xticks(x1)
ax1.set_xticklabels(squad, minor=False, rotation=45)
# fig = plt.figure(figsize = (5, 5))
Precision = [91.222, 93.055, 93.722, 91.222, 90.166, 92.703]

# plt.plot(Accuracy, label = "Accuracy")
plt.bar(squad, Precision)
plt.ylim(90,94)
plt.xlabel('Proposed Approaches')
plt.ylabel('Precision')



fig, ax1 = plt.subplots(1,1, figsize=(6,5))

x1 = [0,1,2,3,4,5]
squad = ['Accuracy-based','Precision-based','F1-based','Recall-based', 'Specificity-based', 'Performance-based']

ax1.set_xticks(x1)
ax1.set_xticklabels(squad, minor=False, rotation=45)
# fig = plt.figure(figsize = (5, 5))
Precision = [93.774, 94.713, 95.095, 93.77, 93.222, 94.111]

# plt.plot(Accuracy, label = "Accuracy")
plt.bar(squad, Precision)
plt.ylim(93,96)
plt.xlabel('Proposed Approaches')
plt.ylabel('F1-score')



fig, ax1 = plt.subplots(1,1, figsize=(6,5))

x1 = [0,1,2,3,4,5]
squad = ['Accuracy-based','Precision-based','F1-based','Recall-based', 'Specificity-based', 'Performance-based']

ax1.set_xticks(x1)
ax1.set_xticklabels(squad, minor=False, rotation=45)
# fig = plt.figure(figsize = (5, 5))
Precision = [96.474, 96.431, 96.510, 96.474, 96.492, 96.443]

# plt.plot(Accuracy, label = "Accuracy")
plt.bar(squad, Precision)
plt.ylim(96,97)
plt.xlabel('Proposed Approaches')
plt.ylabel('Recall')



fig, ax1 = plt.subplots(1,1, figsize=(6,5))

x1 = [0,1,2,3,4,5]
squad = ['Accuracy-based','Precision-based','F1-based','Recall-based', 'Specificity-based', 'Performance-based']

ax1.set_xticks(x1)
ax1.set_xticklabels(squad, minor=False, rotation=45)
# fig = plt.figure(figsize = (5, 5))
Precision = [90.439, 90.435, 90.452, 90.445, 90.425, 90.439]

# plt.plot(Accuracy, label = "Accuracy")
plt.bar(squad, Precision)
plt.ylim(90,90.5)
plt.xlabel('Proposed Approaches')
plt.ylabel('Specificity')



fig, ax1 = plt.subplots(1,1, figsize=(6,5))

x1 = [0,1,2,3,4,5]
squad = ['Accuracy-based','Precision-based','F1-based','Recall-based', 'Specificity-based', 'Performance-based']

ax1.set_xticks(x1)
ax1.set_xticklabels(squad, minor=False, rotation=45)
# fig = plt.figure(figsize = (5, 5))
Precision = [93.774, 94.713, 95.095, 93.77, 93.222, 94.111]

# plt.plot(Accuracy, label = "Accuracy")
plt.bar(squad, Precision)
plt.ylim(93,96)
plt.xlabel('Proposed Approaches')
plt.ylabel('Average Performance')


In [6]:
import matplotlib.pyplot as plt

x=['one', 'two', 'three', 'four', 'five']

# giving the values against
# each value at x axis
y=[91,92,93,94,95]
plt.bar(x, y)

# setting x-label as pen sold
plt.xlabel("pen sold")

# setting y_label as price
plt.ylabel("price")
plt.title(" Vertical bar graph")
plt.ylim(90,100)
plt.show()


In [1]:
 import matplotlib.pyplot as plt
# fig = plt.figure(figsize = (8, 20))
fig, ax1 = plt.subplots(1,1, figsize=(6,5))

x1 = [0,1,2,3,4,5]
squad = ['Accuracy-based','Precision-based','F1-based','Recall-based', 'Specificity-based', 'Performance-based']

ax1.set_xticks(x1)
ax1.set_xticklabels(squad, minor=False, rotation=45)
# fig = plt.figure(figsize = (5, 5))
Accuracy = [93.099, 93.664, 93.625, 92.92, 93.209, 93.324]

plt.plot(Accuracy, label = "Accuracy")
plt.xlabel('Proposed Approaches')
plt.ylabel('Accuracy')


fig, ax1 = plt.subplots(1,1, figsize=(6,5))

x1 = [0,1,2,3,4,5]
squad = ['Accuracy-based','Precision-based','F1-based','Recall-based', 'Specificity-based', 'Performance-based']

ax1.set_xticks(x1)
ax1.set_xticklabels(squad, minor=False, rotation=45)
# fig = plt.figure(figsize = (5, 5))
pr = [91.222, 93.055, 93.722, 91.222, 90.166, 92.703]

plt.plot(pr, label = "Precision")
plt.xlabel('Proposed Approaches')
plt.ylabel('Precision')




fig, ax1 = plt.subplots(1,1, figsize=(6,5))

x1 = [0,1,2,3,4,5]
squad = ['Accuracy-based','Precision-based','F1-based','Recall-based', 'Specificity-based', 'Performance-based']

ax1.set_xticks(x1)
ax1.set_xticklabels(squad, minor=False, rotation=45)
# fig = plt.figure(figsize = (5, 5))
f1 = [93.774, 94.713, 95.095, 93.77, 93.222, 94.111]

plt.plot(f1, label = "F1-Score")
plt.xlabel('Proposed Approaches')
plt.ylabel('F1-Score')



fig, ax1 = plt.subplots(1,1, figsize=(6,5))
x1 = [0,1,2,3,4,5]
squad = ['Accuracy-based','Precision-based','F1-based','Recall-based', 'Specificity-based', 'Performance-based']

ax1.set_xticks(x1)
ax1.set_xticklabels(squad, minor=False, rotation=45)
# fig = plt.figure(figsize = (5, 5))
re = [96.474, 96.431, 96.510, 96.474, 96.492, 96.443]

plt.plot(re, label = "F1-Score")
plt.xlabel('Proposed Approaches')
plt.ylabel('Recall')



fig, ax1 = plt.subplots(1,1, figsize=(6,5))
x1 = [0,1,2,3,4,5]
squad = ['Accuracy-based','Precision-based','F1-based','Recall-based', 'Specificity-based', 'Performance-based']

ax1.set_xticks(x1)
ax1.set_xticklabels(squad, minor=False, rotation=45)
# fig = plt.figure(figsize = (5, 5))
sp = [90.439, 90.435, 90.452, 90.445, 90.425, 90.439]

plt.plot(sp, label = "Specificity")
plt.xlabel('Proposed Approaches')
plt.ylabel('Specificity')


fig, ax1 = plt.subplots(1,1, figsize=(6,5))
x1 = [0,1,2,3,4,5]
squad = ['Accuracy-based','Precision-based','F1-based','Recall-based', 'Specificity-based', 'Performance-based']

ax1.set_xticks(x1)
ax1.set_xticklabels(squad, minor=False, rotation=45)
# fig = plt.figure(figsize = (5, 5))
av = [93.002, 93.659, 93.881, 92.966, 92.703]

plt.plot(av, label = "Average Performance")
plt.xlabel('Proposed Approaches')
plt.ylabel('Average Performance')

In [ ]:
 import matplotlib.pyplot as plt
# fig = plt.figure(figsize = (8, 20))
fig, ax1 = plt.subplots(1,1, figsize=(6,5))

x1 = [0,1,2,3,4]
squad = ['Accuracy-based','Precision-based','F1-based','Recall-based', 'Specificity-based']

ax1.set_xticks(x1)
ax1.set_xticklabels(squad, minor=False, rotation=45)
# fig = plt.figure(figsize = (5, 5))
Accuracy = [93.099, 93.664, 93.625, 92.92, 93.209]

plt.plot(Accuracy, label = "Accuracy")
plt.xlabel('Proposed Approaches')
plt.ylabel('Accuracy')


fig, ax1 = plt.subplots(1,1, figsize=(6,5))

x1 = [0,1,2,3,4]
squad = ['Accuracy-based','Precision-based','F1-based','Recall-based', 'Specificity-based']

ax1.set_xticks(x1)
ax1.set_xticklabels(squad, minor=False, rotation=45)
# fig = plt.figure(figsize = (5, 5))
pr = [91.222, 93.055, 93.722, 91.222, 90.166]

plt.plot(pr, label = "Precision")
plt.xlabel('Proposed Approaches')
plt.ylabel('Precision')




fig, ax1 = plt.subplots(1,1, figsize=(6,5))

x1 = [0,1,2,3,4]
squad = ['Accuracy-based','Precision-based','F1-based','Recall-based', 'Specificity-based']

ax1.set_xticks(x1)
ax1.set_xticklabels(squad, minor=False, rotation=45)
# fig = plt.figure(figsize = (5, 5))
f1 = [93.774, 94.713, 95.095, 93.77, 93.222]

plt.plot(f1, label = "F1-Score")
plt.xlabel('Proposed Approaches')
plt.ylabel('F1-Score')



fig, ax1 = plt.subplots(1,1, figsize=(6,5))
x1 = [0,1,2,3,4]
squad = ['Accuracy-based','Precision-based','F1-based','Recall-based', 'Specificity-based']

ax1.set_xticks(x1)
ax1.set_xticklabels(squad, minor=False, rotation=45)
# fig = plt.figure(figsize = (5, 5))
re = [96.474, 96.431, 96.510, 96.474, 96.492]

plt.plot(re, label = "F1-Score")
plt.xlabel('Proposed Approaches')
plt.ylabel('Recall')



fig, ax1 = plt.subplots(1,1, figsize=(6,5))
x1 = [0,1,2,3,4]
squad = ['Accuracy-based','Precision-based','F1-based','Recall-based', 'Specificity-based']

ax1.set_xticks(x1)
ax1.set_xticklabels(squad, minor=False, rotation=45)
# fig = plt.figure(figsize = (5, 5))
sp = [90.439, 90.435, 90.452, 90.445, 90.425]

plt.plot(sp, label = "Specificity")
plt.xlabel('Proposed Approaches')
plt.ylabel('Specificity')


fig, ax1 = plt.subplots(1,1, figsize=(6,5))
x1 = [0,1,2,3,4]
squad = ['Accuracy-based','Precision-based','F1-based','Recall-based', 'Specificity-based']

ax1.set_xticks(x1)
ax1.set_xticklabels(squad, minor=False, rotation=45)
# fig = plt.figure(figsize = (5, 5))
av = [93.002, 93.659, 93.881, 92.966, 92.703]

plt.plot(av, label = "Average Performance")
plt.xlabel('Proposed Approaches')
plt.ylabel('Average Performance')

In [ ]:
import os
pred_dir = "../input/segwaste/seg_data/DATASET/R"
import random
pred_files = random.sample(os.listdir(pred_dir),5)
from keras.preprocessing import image
i =1
for f in pred_files:
    path = str(pred_dir+f)
    test_image = image.load_img(path, target_size = (128, 128))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = model1.predict(test_image)
    img = image.load_img(path)
    num_rows = 5
    num_cols = 3
    num_images = num_rows,num_cols
    plt.figure(figsize = (6,3))
    plt.subplot(1,1,i)
    plt.grid(False)
    plt.imshow(img)
    plt.show()
    classes =["N","O","R"]    
    print("The predicted image is :" + str(classes[np.argmax(np.array(result))]))
    
    